<a href="https://colab.research.google.com/github/bonghoa15/gee/blob/master/download_gee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

It is assumed that you are authenticated with Google Earth engine your account. If not, sign up your email with GEE with link:  https://earthengine.google.com/new_signup/

In [1]:
!pip install google-api-python-client

In [0]:
import ee

In [3]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=zJEcaJc1cC_WtzSkxl3KJCAiUT8gorqr6aRCArqL1QU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/ywHYuUOYC3A3bBLfMCigSXCLC_PTXXltw3e1ezyudyZ5WaprPKfTqqw

Successfully saved authorization token.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
%cd 'drive/My Drive'

/content/drive/My Drive


In [0]:
gFolder = 'ee' # create folder if not exist

In [0]:
#border Nigeria
border = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')\
                    .filterMetadata("country_na", "equals", "Nigeria").geometry()

In [0]:
#image/imagecollection
nightlight = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG")\
                                    .sort('system:time_end', False)\
                                    .limit(12) #most recent 12 months
worldpop = ee.ImageCollection("WorldPop/GP/100m/pop")\
                        .filterMetadata("country", "equals", "NGA")\
                        .filterMetadata("year", "equals", 2019) #year 2019

In [0]:
#download image collection
def download_imgcol(imgcol, band,des, aggfunc = 'median',border=border, folder=None, scale = None):
    # # collection metadata
    imagename = imgcol.getInfo()["id"]
    id = f"{imagename}_{band}"

    # first image metadata
    meta = imgcol.first().clip(border).getInfo()["bands"][0]
    crs_transform = str(meta["crs_transform"])
    dimensions = meta["dimensions"]

    # clip border, select one band 
    image = getattr(imgcol, aggfunc)().clip(border)
    image = image.set(dict(id=id, crs_transform=crs_transform))
    image = image.select(band)

    #getinfo
    meta = image.getInfo()
    props = meta["properties"]
    crs_transformation = props.get('crs_transform')
    crs = meta['bands'][0]['crs']

    #download 
    task = ee.batch.Export.image.toDrive(image = image,
                                         folder = folder,
                                         description = des,
                                         scale= scale, 
                                         fileFormat= 'GeoTIFF',
                                         maxPixels=int(1e10));
                      
    task.start()
    return print(task.status())

In [10]:
download_imgcol(imgcol=nightlight,des = 'nighttimelight_median', folder = gFolder, band = 'avg_rad',scale=10)

{'state': 'READY', 'description': 'nighttimelight_median', 'creation_timestamp_ms': 1587277749386, 'update_timestamp_ms': 1587277749386, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'OFRVEE4KB4URDZMN54VDRZZJ', 'name': 'projects/earthengine-legacy/operations/OFRVEE4KB4URDZMN54VDRZZJ'}


In [11]:
download_imgcol(imgcol=worldpop, des = 'worldpop_2019',folder = gFolder, band = 'population',scale=10)

{'state': 'READY', 'description': 'worldpop_2019', 'creation_timestamp_ms': 1587277753873, 'update_timestamp_ms': 1587277753873, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'TR7C6Q6JPTIVFJRMAYPXD6CM', 'name': 'projects/earthengine-legacy/operations/TR7C6Q6JPTIVFJRMAYPXD6CM'}
